In [1]:
import argparse
import os
from util import *
import tensorflow as tf
# import keras as K
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers

2022-09-25 21:46:06.883745: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-25 21:46:07.110129: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-25 21:46:07.679689: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-25 21:46:07.679756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [ ]:
# installation
! pip install onnx-tf
! git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow
! pip install -e .

In [ ]:
import onnx
onnx_model_path = "vgg300_1724_3by1256f_beta.onnx"
onnx_model = onnx.load(onnx_model_path)

In [ ]:
from onnx_tf.backend import prepare

tf_rep = prepare(onnx_model)

In [ ]:
tf_model_path = "/home/yeq6/Research_project/Tinyml_contest_2022_UCI/tensorflow_model/vgg300_1724_3by1256f_beta"
tf_rep.export_graph(tf_model_path)

In [2]:

model = tf.saved_model.load(tf_model_path)
model.trainable = False

2022-09-25 21:46:09.295095: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-25 21:46:11.385072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21676 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:73:00.0, compute capability: 8.6


In [4]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
tflite_model = converter.convert()
tflite_model_path = '/home/yeq6/Research_project/Tinyml_contest_2022_UCI/tflite_model/tflite_test'
# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

Estimated count of arithmetic ops: 0.143 M  ops, equivalently 0.071 M  MACs


2022-09-25 21:47:35.237621: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-09-25 21:47:35.237652: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-09-25 21:47:35.237823: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/yeq6/Research_project/Tinyml_contest_2022_UCI/tensorflow_model/vgg300_1724_3by1256f_beta
2022-09-25 21:47:35.238946: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-09-25 21:47:35.238966: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /home/yeq6/Research_project/Tinyml_contest_2022_UCI/tensorflow_model/vgg300_1724_3by1256f_beta
2022-09-25 21:47:35.242376: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-09-25 21:47:35.263221: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /

In [6]:
# TFLite Model Inference
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path= tflite_model_path + ".tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[ 33.719063 -33.635517]]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
